# Trabalho 1 - Modelagem e Avaliação de Desempenho

* Francisco Taam - 119150156
* Felipe Vidal Cantor Bourbon - 119166775
* Lucas Araujo Carvalho - 1191187632
* Lucas Moreno Silva - 11940949
* Pedro Ancelmo de Aquino Vieira Amaral - 119189189

enunciado do trabalho [neste link](https://www.overleaf.com/read/jrykyybwrvsb)

In [1]:
# Import helpers
from helpers import *

## Main code

In [185]:
def simulate(_lambda, _mu, total_time = 10, total_rounds = 1):
  forest = []

  for _ in range(total_rounds):
    event_queue = Queue()
    tree = Tree()

    # Initialize the first events
    arrival_time = arrival_dist(_lambda)
    departure_time = departure_dist(_mu)
    event_queue.push((TYPES[0], arrival_time))
    event_queue.push((TYPES[1], departure_time))

    # Main loop
    while True:
      event = event_queue.pop() # takes the next event from the queue
      tree.current_time = event[1] # gets the time of the event

      if tree.current_time > total_time:
        break

      if event[0] == TYPES[0]:
        tree.arrive_client()
        arrival_time = tree.current_time + arrival_dist(_lambda)
        event_queue.push((TYPES[0], arrival_time))
      elif event[0] == TYPES[1]:
        tree.next_server()
        departure_time = tree.current_time + departure_dist(_mu)
        event_queue.push((TYPES[1], departure_time))

      if tree.current == None:
        print("Tree died\n")
        break

    forest.append(tree)

  return forest


In [184]:
LAMBDA = 1
MU = 2
TOTAL_TIME = 10
TOTAL_ROUNDS = 1

forest = simulate(LAMBDA, MU, TOTAL_TIME, TOTAL_ROUNDS)


Tree died

0.07616507991386015
